In [70]:
#imports

# keras imports
from keras.layers import Embedding, SimpleRNN, Flatten, Dense
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.datasets import imdb
import numpy as np
import keras.backend as K
import keras
import os

# general imports
from IPython.display import display, Markdown #just to display markdown


In [71]:
display(Markdown("## USING IMDB DATA FOR MODEL TRAINING "))

# pre-processing initializations
max_features = 10000  # number of words to consider as features
maxlen = 50  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

# data pre-prcessing
### IMDB data preparation was extensively cover in "text_pre-processing_basic model_building"(earlier module) ###


print('Loading data...')
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=max_features)

# cutting sentences to max length of 50
input_train = sequence.pad_sequences(input_train, maxlen=maxlen)
input_test = sequence.pad_sequences(input_test, maxlen=maxlen)

print ("after data preprocessing")
print('input_train shape:', input_train.shape)
print('input_test shape:', input_test.shape, "\n")

print("\nImdb review 1 sample input data\n")
print(input_train[:1]) 

print("\nImdb review data lables")
print(y_train[:2])
print(
"""
Note:  
0 : "Negative review"
1 : "Positive review"
post_padding in input data helps gragient in LSTM flow better 

"""
)

## USING IMDB DATA FOR MODEL TRAINING 

Loading data...
after data preprocessing
input_train shape: (25000, 50)
input_test shape: (25000, 50) 


Imdb review 1 sample input data

[[2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
    26  480    5  144   30 5535   18   51   36   28  224   92   25  104
     4  226   65   16   38 1334   88   12   16  283    5   16 4472  113
   103   32   15   16 5345   19  178   32]]

Imdb review data lables
[1 0]

Note:  
0 : "Negative review"
1 : "Positive review"
post_padding in input data helps gragient in LSTM flow better 




In [72]:
keras.callbacks.ModelCheckpoint
keras.callbacks.EarlyStopping
keras.callbacks.LearningRateScheduler
keras.callbacks.ReduceLROnPlateau
keras.callbacks.CSVLogger

keras.callbacks.CSVLogger

In [73]:
display(Markdown("## CALLING ALL THE PRE-DEFINED CALLBACKS FOR MODEL TRAINING"))

# This callback will save all the training logs in a file after every epoch
dir_path = "./"
csv_logger = keras.callbacks.CSVLogger(
    # This is the path to the file where logs data will be stored
    os.path.join(dir_path,"training.log")
)


# This callback will interrupt training when we have stopped improving
early_stopping = keras.callbacks.EarlyStopping(
    # This callback will monitor the validation accuracy of the model
    monitor='acc',
    # Training will be interrupted when the accuracy
    # has stopped improving for *more* than 3 epochs
    patience=4,
)

# This callback will save the current weights after every epoch
# The name of weight file contains epoch number, val accuracy
file_path="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoints = keras.callbacks.ModelCheckpoint(
        filepath=file_path,  # Path to the destination model file
        # The two arguments below mean that we will not overwrite the
        # model file unless `val_loss` has improved, which
        # allows us to keep the best model every seen during training.
        monitor='val_loss',
        save_best_only=True,
    )

reduce_learning_rate = keras.callbacks.ReduceLROnPlateau(
    # This callback will monitor the validation loss of the model
    monitor='val_loss',
    # It will divide the learning by 10 when it gets triggered
    factor=0.1,
    # It will get triggered after the validation loss has stopped improving
    # for at least 3 epochs
    patience=3,
    # Note that since the callback will be monitor validation loss,
    # we need to pass some `validation_data` to our call to `fit`.
    )

## CALLING ALL THE PRE-DEFINED CALLBACKS FOR MODEL TRAINING

In [101]:
display(Markdown("## WRITING A CUSTOM CALLING FOR TEST EVALUATION DURING TRAINING"))

# Called at the start of every epoch
# -->on_epoch_begin

# Called at the end of every epoch
# -->on_epoch_end

# Called right before processing each batch
# -->on_batch_begin

# Called right after processing each batch
# -->on_batch_end

# Called at the start of training
# -->on_train_begin

# Called at the end of training
# -->on_train_end

class TestAccuracyCalculator(keras.callbacks.Callback):
    
    def __init__(self, test_data=None,test_lables=None):
        # Passing test data separately
        # to test accuracy changes while training
        self.test_data = test_data
        self.test_lables = test_lables
        
    def set_model(self, model):
        # This method is called by the parent model
        # before training, to inform the callback
        # of what model will be calling it
        self.model = model
    
    def calculate_binary_accuracy(self):
        pass
    
    def on_epoch_end(self, epoch, logs=None):
    
        if (type(self.test_data) !=  type(None)) and (type(self.test_lables) !=  type(None)):
            # Using current best weight from this epoch
            # to test on provided test data-set
            y_pred = self.model.predict(self.test_data)
            y_pred_ = np.where(y_pred > 0.5, 1, 0)


            # Accuracy calculation for binary predictions
#             accuracy = (K.mean(K.equal(self.test_lables, K.round(y_pred))))
            accuracy = np.sum(self.test_lables == y_pred_)
            print (y_pred_.shape)
            print (accuracy)
#             print ("Test accuracy after {} epocs is {}%".format(epoch,accuracy))
        else:
            pass

test_accuracy_calculator = TestAccuracyCalculator(test_data=input_test, test_lables=y_test)

print ("""
Here is a simple example of a custom callback, where we save to disk (as Numpy arrays) 
the activations of every layer of the model at the end of every epoch, computed on the 
first sample of the validation set.
""")

## WRITING A CUSTOM CALLING FOR TEST EVALUATION DURING TRAINING


Here is a simple example of a custom callback, where we save to disk (as Numpy arrays) 
the activations of every layer of the model at the end of every epoch, computed on the 
first sample of the validation set.



In [102]:
# Callbacks are passed to the model fit the `callbacks` argument in `fit`,
# which takes a list of callbacks. You can pass any number of callbacks.
callbacks_list = [ csv_logger, early_stopping,
                  checkpoints, reduce_learning_rate,
                  test_accuracy_calculator
                 ]

In [103]:
display(Markdown("## TRAINING MODEL WITH CALLBACKS "))

# model architecture and training
print("starting model training...")
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))
model.summary()
print ("\n")
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(input_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=callbacks_list)

## TRAINING MODEL WITH CALLBACKS 

starting model training...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_28 (Embedding)     (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_28 (SimpleRNN)    (None, 32)                2080      
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 33        
Total params: 322,113
Trainable params: 322,113
Non-trainable params: 0
_________________________________________________________________


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 11s 538us/step - loss: 0.5453 - acc: 0.7198 - val_loss: 0.5046 - val_acc: 0.7590
(25000, 1)
312500000
Epoch 2/10
20000/20000 [==============================] - 9s 440us/step - loss: 0.3701 - acc: 0.8417 - val_loss: 0.4275 - val_acc: 0.8060
(25000, 1)
312500000
Epo

In [ ]:
9*150